In [7]:
import os
import sys
import shutil
import getpass
import codecs

repos_dir = r'/home/akikun/repos'
sys.path.append(repos_dir)
from sak import display as dp
from rs_corpus import jp

password = 'guoyue21'

In [2]:
def tts_hikari(text_to_read, wav_path, voice='Hikari', vt_dir=r'/usr/vt'):
    '''
    perform TTS using vtspeech. 
    The output file will be made in {vt_dir}/result/vthenrik/{output_filename}.wav
    
    Args:
        text_to_read (str): text to be read.
        wav_path (path): the output file. 
        voice (str): which voice to be used. default: Hikari.
        vt_dir (path): where vtspeech is installed. default: /usr/vt.
        
    '''
    
    # make tts sample.
    wav_filename = os.path.basename(wav_path).replace('.wav', '')
    command = 'sudo -S ' + os.path.join(vt_dir, 'rest', 'vthenrik') \
    + ' --lang ja' \
    + ' --voice ' + voice \
    + ' --text ' + text_to_read \
    + ' --afile ' + wav_filename

    #password = getpass.getpass()
    #os.system('echo %s | %s' % (password, command))
    !echo {password} | {command}
    
    # copy the wav file to the desired location.
    wav_path_ = os.path.join(vt_dir, 'result', 'vthenrik', os.path.basename(wav_path))
    shutil.move(wav_path_, wav_path)

In [3]:
def loop_per_sentence(txt_path, trs_path):
    ''' make a loop per sentences using str and the corresponding trs '''
    
    # read trans.
    with open(trs_path) as f:
        trss = f.read().split('\n')
        trss.remove('')
    len(trss)

    # read text.
    txts = codecs.open(txt_path,"rb",'shift_jisx0213').read().split('\r\n')

    # show script and transcription per sentence.
    trs_ = '' # transcription.
    txt_ = '' # script.
    for i, trs in enumerate(trss):
#         if i > 10:
#             break

        txt = txts[i]
        trs_ += trs
        txt_ += txt
        if '。' in txt:
            yield txt_, trs_
            trs_ = ''
            txt_ = ''
            

def check_charactors(txt):
    ng_charactors = ['(', ')', '<', '>', '&']
    check = [1 for i in ng_charactors if i in txt]
    is_ok = True
    if len(check) > 0:
        is_ok = False
    return is_ok

## make tts samples for iMetricGAN training.

In [5]:
tts_dir  = r'/home/akikun/Projects/nele/hikari_tts'
hikari_dir = r'/home/akikun/Corpus/Japanese/02_USS/Female/04_Hikari'

In [6]:
# split into sentences.
# for i, txt_ in enumerate(txts.split('。'), start=1):
#     i_str = format(i, '03d')
#     txt = txt_.replace('\r', '').replace('\n', '') + '。'
#     if len(txt) > 1:
#         print('{}:{}'.format(i_str, txt))

In [11]:
filelist = [file.replace('.txt', '') for file in os.listdir(os.path.join(hikari_dir, 'text')) if len(file) == 14]
filelist.sort()
for filename in filelist:
    #print('\n>>> processing {} ...'.format(filename))
    #filename = 'hikari-001'
    txt_path = os.path.join(hikari_dir, 'text', filename + '.txt')
    trs_path = os.path.join(hikari_dir, 'trans', filename + '.trs')

    i = 0
    error_list = []
    for txt, trs in loop_per_sentence(txt_path, trs_path):
        i += 1
        i_str = format(i, '03d')
        basename = filename + '_' + i_str
#         print('=== {} ==='.format(basename))
#         print('{}'.format(txt))
        
        if not check_charactors(txt):
            print('=== {} ==='.format(basename))
            print('{}'.format(txt))
        
#         if check_charactors(txt):
#             tts_hikari(txt, os.path.join(tts_dir, 'wavs', basename + '.wav'))
#             with open(os.path.join(tts_dir, 'scripts', basename + '.txt'), 'w') as f:
#                 f.write(txt)    
#             with open(os.path.join(tts_dir, 'scripts', basename + '.trs'), 'w') as f:
#                 f.write(trs)
#         else:
#             error_list.append(basename)
#             print('+++++ cannot generate audio +++++')

=== hikari-029_007 ===
東日本大震災の発生から2ヵ月、国境なき医師団(MSF)の臨床心理士チームは、引き続き、被災者への心理ケアを行っている。
=== hikari-029_047 ===
放射性ヨウ素131は甲状腺に取り込まれ、様々な甲状腺障害、(甲状腺機能低下症、甲状腺ガン)を引き起こします。
=== hikari-029_077 ===
MOX燃料は、ウランとプルトニウムを酸化物の形で混合した燃料で、ウラン燃料同様、陶器のように焼き固め安定した状態(燃料ペレット)で使用されます。
=== hikari-031_050 ===
茨城県東海村にある動燃(動力炉・核燃料開発事業団)核廃棄物再処理施設内にて、重大な事故が発生した。
=== hikari-033_086 ===
菅直人前首相が今年春のOECD(経済協力開発機構)の総会や、G8サミット(主要国首脳会議)の場で表明した、再生可能エネルギーの普及目標作りが焦点だ。
=== hikari-033_087 ===
20カ国・地域(G20)ならびに主要8カ国(G8)議長国でもあるだけに、サルコジ大統領はギリシャなどへ手を差し伸べることに前向きだ。
=== hikari-034_003 ===
アメリカのオバマ大統領が初めて出席する東アジアサミットが行われる予定だが、アメリカが主導するTPP(環太平洋戦略的経済連携協定)に中国が警戒感を示していて、経済連携をめぐって、野田首相が難しい対応を迫られる局面も予想される。
=== hikari-041_023 ===
1980/5/25(月)からちょうど十年後、インドでラジブ・ガンジーが第七代首相になりました。
=== hikari-041_025 ===
2004/12/3(水)からちょうど十年前、プレイステーションが発売されました。
=== hikari-041_026 ===
彼女に初めて会ったのは1936年8月31日(土)です。
=== hikari-041_027 ===
私が30歳になるのは2021.5.21(月)です。
=== hikari-041_028 ===
平成16年1月10日(木)、日本数学検定協会が数学の日に制定しました。
=== hikari-041_029 ===
昭和52年2月11日(水)の約四ヶ月後、盧溝橋事件が起こり

In [8]:
error_list

[]

## make demo samples for JR.

In [13]:
wav_dir  = r'/home/akikun/Projects/nele/hikari_tts/demo'
wav_path = os.path.join(wav_dir, 'hikari_3.wav')
tts_hikari(
     #'まもなく３番線に、のぞみ3号博多行きが参ります。危ないので、白線の内側までお下がりください。', 
    #'JR東海をご利用くださいまして、ありがとうございます。',
    'この電車は、のぞみ3号博多行きです。グリーン車は4号車と5号車です。',
    wav_path
     )
dp.disp_wav(wav_path)

[sudo] password for akikun: Normal file mode. Using encoding: 2
Successfully created file hikari_3.wav
